In [1]:
import pandas as pd

# Replace 'file_path.csv' with the path to your CSV file
file_path = './zomato.csv'

# Read the CSV file into a pandas DataFrame, specifying the encoding
df = pd.read_csv(file_path, encoding='latin1')

# Now you can work with the DataFrame 'df'
df = df[df['Country Code'] == 1]

In [34]:
from geopy.distance import geodesic
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd

class RestaurantRecommender:
    def __init__(self, df):
        self.df = df
        self.tfidf_vectorizer = TfidfVectorizer()
        
    def recommend_restaurants_custom_row(self, custom_row, max_distance_km=100):
        within_range_restaurants = self.df[
            self.df.apply(
                lambda x: geodesic((x['Latitude'], x['Longitude']), (custom_row['Latitude'], custom_row['Longitude'])).kilometers <= max_distance_km,
                axis=1
            )
        ]
        
        within_budget_restaurants = within_range_restaurants[
            within_range_restaurants['Average Cost for two'] <= custom_row['Average Cost for two']
        ]
        
        return within_budget_restaurants

    def recommend_similar_cuisines(self, custom_row):
        within_budget_restaurants = self.recommend_restaurants_custom_row(custom_row)
        
        if within_budget_restaurants.empty:
            return "No restaurants available"
        
        tfidf_matrix = self.tfidf_vectorizer.fit_transform(within_budget_restaurants['Cuisines'])
        custom_cuisine_tfidf = self.tfidf_vectorizer.transform([custom_row['Cuisines']])
        
        cosine_similarities = linear_kernel(custom_cuisine_tfidf, tfidf_matrix).flatten()
        
        same_cuisine_indices = [i for i, sim in sorted(enumerate(cosine_similarities), key=lambda x: x[1], reverse=True) if any(cuisine.strip() == custom_row['Cuisines'] for cuisine in within_budget_restaurants.iloc[i]['Cuisines'].split(','))]
        
        similar_cuisine_indices = [i for i, sim in sorted(enumerate(cosine_similarities), key=lambda x: x[1], reverse=True) if sim > 0.5]
        
        same_cuisine_restaurants = within_budget_restaurants.iloc[same_cuisine_indices[:10]]
        similar_cuisine_restaurants = within_budget_restaurants.iloc[similar_cuisine_indices[:10]]
        
        recommended_restaurants = pd.concat([same_cuisine_restaurants, similar_cuisine_restaurants])
        return recommended_restaurants if not recommended_restaurants.empty else "No restaurants available"

recommender = RestaurantRecommender(df)

custom_row = {
    'Longitude': 80.201859,
    'Latitude': 13.072090,
    'Cuisines': 'Continental',
    'Average Cost for two': 2000,
}

recommended_restaurants = recommender.recommend_similar_cuisines(custom_row)
recommended_restaurants

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
821,65055,Barbeque Nation,1,Chennai,"Shri Devi Park Hotel, 1, Hanumantha Road, Nort...",T. Nagar,"T. Nagar, Chennai",80.233110,13.045479,"North Indian, Continental",...,Indian Rupees(Rs.),No,No,No,No,4,4.4,Green,Very Good,3848
812,71492,Ciclo Cafe,1,Chennai,"47, Gandhi Mandapam Road, Kotturpuram, Chennai",Kotturpuram,"Kotturpuram, Chennai",80.242439,13.022394,"Cafe, Continental",...,Indian Rupees(Rs.),No,No,No,No,3,4.1,Green,Very Good,1004
807,70431,Pantry d'or,1,Chennai,"21/11, J Block, 6th Avenue Main Road, Anna Nag...",Anna Nagar East,"Anna Nagar East, Chennai",80.219104,13.091809,"Continental, Cafe, Italian, Desserts",...,Indian Rupees(Rs.),Yes,Yes,No,No,2,4.4,Green,Very Good,1504
820,70497,Basil With A Twist,1,Chennai,"58-A, Habibullah Road, T. Nagar, Chennai",T. Nagar,"T. Nagar, Chennai",80.242268,13.049645,"Continental, Cafe, Spanish, Italian, European,...",...,Indian Rupees(Rs.),Yes,Yes,No,No,3,4.5,Dark Green,Excellent,1210
821,65055,Barbeque Nation,1,Chennai,"Shri Devi Park Hotel, 1, Hanumantha Road, Nort...",T. Nagar,"T. Nagar, Chennai",80.233110,13.045479,"North Indian, Continental",...,Indian Rupees(Rs.),No,No,No,No,4,4.4,Green,Very Good,3848
812,71492,Ciclo Cafe,1,Chennai,"47, Gandhi Mandapam Road, Kotturpuram, Chennai",Kotturpuram,"Kotturpuram, Chennai",80.242439,13.022394,"Cafe, Continental",...,Indian Rupees(Rs.),No,No,No,No,3,4.1,Green,Very Good,1004


In [12]:
from geopy.distance import geodesic
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import json

class BudgetHandler:
    def filter_by_budget(self, df, budget):
        return df[df['Average Cost for two'] <= budget]

class LocationHandler:
    def filter_by_location(self, df, latitude, longitude, max_distance_km):
        return df[df.apply(lambda x: geodesic((x['Latitude'], x['Longitude']), (latitude, longitude)).kilometers <= max_distance_km, axis=1)]

class CuisineHandler:
    def filter_by_cuisine(self, df, cuisine):
        return df[df['Cuisines'].str.contains(cuisine)]

class RestaurantRecommender:
    def __init__(self, df):
        self.df = df
        self.tfidf_vectorizer = TfidfVectorizer()
        self.budget_handler = BudgetHandler()
        self.location_handler = LocationHandler()
        self.cuisine_handler = CuisineHandler()
        
    def recommend_restaurants(self, custom_row):
        latitude = custom_row.get('latitude')
        longitude = custom_row.get('longitude')
        cuisine = custom_row.get('cuisine')
        budget = custom_row.get('budget')
        max_distance_km = custom_row.get('max_distance_km', 100)  # Default max distance is 100 km
        
        budget_filtered = self.budget_handler.filter_by_budget(self.df, budget)
        location_filtered = self.location_handler.filter_by_location(budget_filtered, latitude, longitude, max_distance_km)
        cuisine_filtered = self.cuisine_handler.filter_by_cuisine(location_filtered, cuisine)
        return cuisine_filtered

    def recommend_similar_cuisines(self, custom_row):
        latitude = custom_row.get('latitude')
        longitude = custom_row.get('longitude')
        cuisine = custom_row.get('cuisine')
        budget = custom_row.get('budget')
        max_distance_km = custom_row.get('max_distance_km', 100)  # Default max distance is 100 km
        
        restaurants = self.recommend_restaurants(custom_row)
        
        if restaurants.empty:
            return "No restaurants available"
        
        tfidf_matrix = self.tfidf_vectorizer.fit_transform(restaurants['Cuisines'])
        custom_cuisine_tfidf = self.tfidf_vectorizer.transform([cuisine])
        
        cosine_similarities = linear_kernel(custom_cuisine_tfidf, tfidf_matrix).flatten()
        
        similar_cuisine_indices = [i for i, sim in sorted(enumerate(cosine_similarities), key=lambda x: x[1], reverse=True) if sim > 0.1]
        
        similar_cuisine_restaurants = restaurants.iloc[similar_cuisine_indices[:10]]
        
        # Convert recommended restaurants DataFrame to a list of dictionaries
        similar_cuisine_restaurants = similar_cuisine_restaurants.to_dict(orient="records")
        
        return similar_cuisine_restaurants

# Example usage:
custom_row = {
    'latitude': 13.072090,
    'longitude': 80.201859,
    'cuisine': 'Continental',
    'budget': 2000,
    'max_distance_km': 100  # Adjust as needed
}

# Initialize RestaurantRecommender with your DataFrame
recommender = RestaurantRecommender(df)

# Call the method with the custom_row dictionary
recommended_restaurants = recommender.recommend_similar_cuisines(custom_row)
recommended_restaurants

[{'Restaurant ID': 71492,
  'Restaurant Name': 'Ciclo Cafe',
  'Country Code': 1,
  'City': 'Chennai',
  'Address': '47, Gandhi Mandapam Road, Kotturpuram, Chennai',
  'Locality': 'Kotturpuram',
  'Locality Verbose': 'Kotturpuram, Chennai',
  'Longitude': 80.24243889,
  'Latitude': 13.02239444,
  'Cuisines': 'Cafe, Continental',
  'Average Cost for two': 1100,
  'Currency': 'Indian Rupees(Rs.)',
  'Has Table booking': 'No',
  'Has Online delivery': 'No',
  'Is delivering now': 'No',
  'Switch to order menu': 'No',
  'Price range': 3,
  'Aggregate rating': 4.1,
  'Rating color': 'Green',
  'Rating text': 'Very Good',
  'Votes': 1004},
 {'Restaurant ID': 65055,
  'Restaurant Name': 'Barbeque Nation',
  'Country Code': 1,
  'City': 'Chennai',
  'Address': 'Shri Devi Park Hotel, 1, Hanumantha Road, North Usman Road, T. Nagar, Chennai',
  'Locality': 'T. Nagar',
  'Locality Verbose': 'T. Nagar, Chennai',
  'Longitude': 80.23311,
  'Latitude': 13.045479,
  'Cuisines': 'North Indian, Continen